In [1]:
from scipy.io import loadmat
lfp_buz_features = loadmat('lfp_buz_features_rgs14.mat')

## EEG Bands and EMG Like Signals

In [2]:
import pandas as pd
import numpy as np

lfp_matrix = np.array(lfp_buz_features['lfpFeatures'])

numColumns = len(lfp_matrix[0])

delta_pfc_df = pd.DataFrame(lfp_matrix[:, 0], columns=["DeltaPFC"])
theta_hpc_df = pd.DataFrame(lfp_matrix[:, 1], columns=["ThetaHPC"])
beta_pfc_df = pd.DataFrame(lfp_matrix[:, 2], columns=["BetaPFC"])
gamma_hpc_df = pd.DataFrame(lfp_matrix[:, 3], columns=["GammaHPC"])
emg_like_df = pd.DataFrame(lfp_matrix[:, 4], columns=["EMGLike"])

timesteps = 10541

In [3]:
delta_pfc = delta_pfc_df['DeltaPFC'].values
theta_hpc  = theta_hpc_df['ThetaHPC'].values

delta_pfc = delta_pfc[delta_pfc != 0]
theta_hpc = theta_hpc[theta_hpc != 0]

delta_pfc_by_theta_hpc = delta_pfc / theta_hpc

emg_like = emg_like_df.EMGLike.values
emg_like = emg_like[emg_like != 0]

In [4]:
type(delta_pfc_by_theta_hpc)

numpy.ndarray

In [5]:
delta_pfc_by_theta_hpc.shape

(10541,)

In [6]:
delta_pfc.shape

(10541,)

In [7]:
theta_hpc.shape

(10541,)

In [8]:
emg_like.shape

(10541,)

## Bands
- Nx4 data array, where N is the number of epochs, and columns refer to Delta PFC, Theta HPC, Delta/Theta and EMG

In [9]:
rgs_14_bands = np.column_stack([
    delta_pfc, 
    theta_hpc, 
    delta_pfc_by_theta_hpc, 
    emg_like, 
])

In [10]:
rgs_14_bands.shape

(10541, 4)

In [11]:
rgs_14_bands

array([[0.14977489, 0.10481071, 1.4290037 , 0.93493003],
       [0.15763617, 0.11268804, 1.39887228, 0.92934066],
       [0.16509083, 0.12095128, 1.36493664, 0.91670823],
       ...,
       [0.38752742, 0.55695679, 0.6957944 , 0.09506661],
       [0.400658  , 0.51908918, 0.7718481 , 0.08132514],
       [0.4145541 , 0.48116965, 0.86155496, 0.07903146]])

In [12]:
np.min(rgs_14_bands, axis=0)

array([0.01607664, 0.00374557, 0.03391385, 0.00016225])

In [13]:
np.max(rgs_14_bands, axis=0)

array([  1.        ,   1.        , 241.40259618,   1.        ])

## EpochsLinked
- Nx4 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: epoch index (currently not used)
	- column 3: ground truth sleep stage ID, where

				- 1 is associated with wakefulness,
				- 2 is associated with NREM sleep,
				- 3 is associated with REM sleep
	- column 4: the subject ID (used in multi-subject analysis only)

In [14]:
epoch_ids = np.arange(timesteps, dtype=int)

epoch_ids.size

10541

In [15]:
epoch_index = np.arange(timesteps, dtype=int)

epoch_index.size

10541

In [16]:
states = loadmat('states.mat')

states.keys()

dict_keys(['__header__', '__version__', '__globals__', 'downsampledStates', 'keys', 'None', '__function_workspace__'])

In [17]:
states['downsampledStates']

array([[0, 0, 0, ..., 4, 4, 4]], dtype=uint8)

In [18]:
ground_truth_sleep_stage_id = states['downsampledStates'].astype(int)

ground_truth_sleep_stage_id = ground_truth_sleep_stage_id.flatten()

In [19]:
ground_truth_sleep_stage_id

array([0, 0, 0, ..., 4, 4, 4])

In [20]:
subject_id = np.ones(timesteps, dtype=int)

subject_id.size

10541

In [21]:
epochs_linked = np.column_stack([epoch_ids, 
                                 epoch_index, 
                                 ground_truth_sleep_stage_id[:10541],
                                 subject_id
                                ])

# epochs_linked.shape

In [22]:
epochs_linked.shape

(10541, 4)

## EpochTime
- Nx3 data array, where N is the number of epochs, and columns are described as follows:

	- column 1: epoch ID
	- column 2: recording mode (i.e. baseline or recovery), where
    
			   - 1 is associated with baseline,
			   - 2 is associated with recovery (after sleep deprivation)
	- column 3: the epoch date-time

In [23]:
recording_mode = np.ones(timesteps).astype('<f8')

In [24]:
start_value = 41137.2918055555
step_size = 8.234392028162487e-05

time = np.arange(start_value, start_value + step_size * timesteps, step_size).astype('<f8')

time = time[:10542]

time.shape

(10541,)

In [25]:
epoch_time = np.column_stack([epoch_ids.astype('<f8'), recording_mode, time])

epoch_time

array([[0.00000000e+00, 1.00000000e+00, 4.11372918e+04],
       [1.00000000e+00, 1.00000000e+00, 4.11372919e+04],
       [2.00000000e+00, 1.00000000e+00, 4.11372920e+04],
       ...,
       [1.05380000e+04, 1.00000000e+00, 4.11381595e+04],
       [1.05390000e+04, 1.00000000e+00, 4.11381596e+04],
       [1.05400000e+04, 1.00000000e+00, 4.11381597e+04]])

In [26]:
np.savez("rgs14.npz", d=rgs_14_bands, epochsLinked=epochs_linked, epochTime=epoch_time)